<hr>

# AI Workflow Capstone - AAVAIL - **Part 3 - Model Production**

Student: `Alexey Tyurin`<br>
Date: `1/18/2024`
<hr>

### Outline

1. Build a draft version of an API with train, predict, and logfile endpoints.
2. Using Docker, bundle your API, model, and unit tests.
3. Using test-driven development iterate on your API in a way that anticipates scale, load, and drift.
4. Create a post-production analysis script that investigates the relationship between model performance and the business metric.
5. Articulate your summarized findings in a final report.

At a higher level you are being asked to:

1. Ready your model for deployment
2. Query your API with new data and test your monitoring tools
3. Compare your results to the gold standard


### Create a flask API

The API performs the train, predict and logfile tasks

In [2]:
#here we will be using the data from the previous notebook to create different models and compare them to see which one is the best

# Importing the libraries
from model import model_train, model_load, model_predict
import requests

In [3]:
model_train('cs-train')

... loading ts data from files
... saving model: models\sl-all-0_1.joblib
... saving model: models\sl-eire-0_1.joblib
... saving model: models\sl-france-0_1.joblib
... saving model: models\sl-germany-0_1.joblib
... saving model: models\sl-hong_kong-0_1.joblib
... saving model: models\sl-netherlands-0_1.joblib
... saving model: models\sl-norway-0_1.joblib
... saving model: models\sl-portugal-0_1.joblib
... saving model: models\sl-singapore-0_1.joblib
... saving model: models\sl-spain-0_1.joblib
... saving model: models\sl-united_kingdom-0_1.joblib


In [4]:
# load the model
all_data, all_models = model_load(data_dir='cs-train')
print("all_data: ", ", ".join(all_models.keys()))

... loading ts data from files
all_data:  all, eire, france, germany, hong_kong, netherlands, norway, portugal, singapore, spain, united_kingdom


In [5]:
## API train
query = {'dev': 'True',
         'verbose': 'True'}

port = 8080
r = requests.post(f'http://localhost:{port}/train', json=query)
pritn(r)

In [7]:
## API logging
query = {'env': 'test',
         'type': 'train',
         'year': '2020',
         'month': '5'}

port = 8080
r = requests.post(f'http://localhost:{port}/logging', json=query)
response = literal_eval(r.text)
print(response)

{'logfile': 'test-train-2020-5.log'}


In [8]:
## API predict
import requests
from ast import literal_eval

query = {'year': '2018',
         'month': '1',
         'day': '5',
         'country': 'Total',
         'dev': 'True',
         'verbose': 'True'}

port = 8080
r = requests.post(f'http://localhost:{port}/predict', json=query)
#response = literal_eval(r.text)
print(r)

<Response [500]>


## Create Unit Tests

The unit tests are usually organized as a suite and return objective evidence, in the form of a boolean value, which is a key element that enables workflow automation. The boolean value indicates whether or not each and every part of the software that was tested performed as expected. Much like data ingestion, the idea is to have the necessary components of a task bundled under a single script. In this case it will be called `run-tests.py`.